<a href="https://colab.research.google.com/github/danyentezari/bn-ocr/blob/main/keras_ocr_realvalues_bignumber.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install keras==2.2.5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 336 kB 24.6 MB/s 
     |████████████████████████████████| 50 kB 6.7 MB/s 
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.2+zzzcolab20220719082949 requires keras<2.9,>=2.8.0rc0, but you have keras 2.2.5 which is incompatible.


In [ ]:
!pip install tensorflow==1.13.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 92.6 MB 65 kB/s 
     |████████████████████████████████| 3.2 MB 44.0 MB/s 
     |████████████████████████████████| 367 kB 7.3 MB/s 
ERROR: Operation cancelled by user


In [ ]:
!pip install 'h5py==2.10.0' --force-reinstall

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.9 MB 24.7 MB/s 
     |████████████████████████████████| 15.7 MB 42.5 MB/s 
  Attempting uninstall: six
    Found existing installation: six 1.15.0
    Uninstalling six-1.15.0:
      Successfully uninstalled six-1.15.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.2+zzzcolab20220719082949 requires keras<2.9,>=2.8.0rc0, but you have keras 2.2.5 which is incompatible.


In [ ]:
import tensorflow as tf
import keras
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np
from os import listdir
from os.path import isfile, join
import re
from scipy.sparse import csr_matrix
import cv2

from sklearn.feature_extraction.text import HashingVectorizer

import string
from PIL import Image
import numpy as np

from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array

import itertools

Using TensorFlow backend.


ModuleNotFoundError: ignored

In [ ]:
def labels_to_text(labels, alphabet):
    """Reverse translation of numerical classes back to characters."""
    ret = []
    for c in labels:
        if c == len(alphabet):  # CTC Blank
            ret.append("")
        else:
            ret.append(alphabet[c])
    return "".join(ret)


def text_to_labels(text, alphabet):
    """ Translation of characters to unique integer values
    """
    ret = []
    for char in text:
        ret.append(alphabet.find(char))
    return ret


def decode_batch(test_func, word_batch, alphabet):
    """
    - Greedy search -
    For a real OCR application, this should be beam
    search with a dictionary and language model.
    For this example, best path is sufficient.
    """
    res = []
    prob = []
    out = test_func([word_batch])[0]
    for i in range(out.shape[0]):
        out_best = list(np.argmax(out[i, 2:], 1))
        out_prob = np.mean(np.max(out[i, 2:], 1))
        out_best = [k for k, g in itertools.groupby(out_best)]
        outstr = labels_to_text(out_best, alphabet)
        res.append(outstr)
        prob.append(out_prob)
    return res, prob


In [ ]:
!pip install gdown

In [ ]:
# # Get features
!gdown https://drive.google.com/uc?id=176PKaCUDWmTJdQwc-OfkO0y8t4gLsIvQ
!unzip formulae.zip

# # Get labels
!gdown https://drive.google.com/uc?id=1QUjX6PFWPa-HBWdcY-7bA5TRVUnbyS1D

In [ ]:
alphabet = string.digits + \
           string.ascii_lowercase + \
           string.ascii_uppercase + \
           '\n' + \
           '\t' + \
           "".join([' ', '~', ':', "'", '+', '[', '\\', '@', '^', '{', '%', '(', '-', '"', '*', '|', ',', '&', '<', '`', '}', '.', '_', '=', ']', '!', '>', ';', '?', '#', '$', ')', '/'])

In [ ]:
len(alphabet)

In [ ]:
def loadImage(filepath):
  img = load_img(filepath)
  img = img_to_array(img)
  # img = img.reshape(1, 448, 64, 3)
  # img = img.reshape(64, 192, 3)
  img = img.astype('float32')
  # img = img / 255.0
  return img


sample_image = loadImage('train/0000003.png')


Image.fromarray(np.uint8(sample_image )).convert('RGB')

In [ ]:
def create_canvas(canvas_h=300, canvas_w=300):
  # Create canvas
  canvas_color = 255

  pixel_rows = []
  row = [canvas_color for x in range(0,canvas_w)]
  for i in range(0,canvas_h):
    pixel_rows.append(row)
  r = pixel_rows
  g = pixel_rows
  b = pixel_rows

  canvas = np.dstack((r,g,b))
  return canvas

cw = 850
ch = 300
canvas = create_canvas(ch, cw)

def center_pad_image(img, canvas_h=300, canvas_w=300):
  # Image positions
  img2_width = img.shape[1]
  img2_height = img.shape[0]
  black_row = np.array([[0 for x in range(0,img2_width)]])
  img2 = np.dstack((black_row,black_row,black_row))


  startx = int((canvas_w-img2_width)/2)
  starty = int((canvas_h-img2_height)/2)
  endx = startx + img2_width
  endy = starty + img2_height

  # Superimpose image
  to_superimpose = Image.fromarray(np.uint8(img)).convert('RGB')
  # Reset global canvas
  canvas[:, :] = 255
  canvas[starty:endy, startx:endx] = to_superimpose


  return Image.fromarray(np.uint8(canvas)).convert('RGB')


center_pad_image(sample_image, ch, cw)

In [ ]:
label_file = open('./math.txt', 'r')
label_classes = [label for label in label_file]

In [ ]:
mypath = './train'
train_filenames = [f for f in listdir(mypath) if isfile(join(mypath, f))]

### Features and Labels

In [ ]:
alphabet_list = list(alphabet)
alphabet_t2id = dict({})
alphabet_id2t = dict({})
for i, achar in enumerate(alphabet_list):
  alphabet_t2id[achar] = i
  alphabet_id2t[i] = achar

In [ ]:
# ch = 100
# cw = 720
# ch = 300
# cw = 780
n = 300
max_word_length_track = 0

train_F = np.empty((n, ch, cw, 3))
train_L = ['' for x in range(0,n)]

canvas = create_canvas(ch, cw)

for i, filname in enumerate(train_filenames[0:n]):
  loaded_image = loadImage(f'train/{filname}')
  loaded_image = np.array(center_pad_image(loaded_image, ch, cw))

  # Normalize
  loaded_image = loaded_image / 255

  # loaded_image = cv2.resize(loaded_image,(64,64))
  train_F[i] = loaded_image
  # train_L.append(int(filname.split('.')[0]))
  # train_L.append(label_classes[int(filname.split('.')[0])])
  train_L[i] = label_classes[int(filname.split('.')[0])]

  if len(train_L[i]) > max_word_length_track:
    max_word_length_track = len(train_L[i])

train_L = np.array(train_L)

In [ ]:
# Preview training image (centered and padded)
Image.fromarray(np.uint8(train_F[0])).convert('RGB')

In [ ]:
print(train_F.shape)
print(train_L.shape)

In [ ]:
train_L_id = np.array([np.zeros(max_word_length_track)])
train_L_t = []

for i,label in enumerate(train_L):
  ids = np.zeros(max_word_length_track)
  new_item = [float(alphabet_t2id[t]) for i,t in enumerate(list(label))]
  ids[0: len(new_item)] = new_item
  train_L_id = np.append(train_L_id, [ids], axis=0)


# train_L_id = np.delete(train_L_id, [0])
train_L_id = train_L_id[1:]
len(train_L_id)

### Model

In [ ]:
!pip install cairocffi

In [ ]:
import os
import itertools
import codecs
import re
import datetime
import cairocffi as cairo
import editdistance
import numpy as np
from scipy import ndimage
import pylab
import tensorflow
from keras import backend as K
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers import Input, Dense, Activation
from keras.layers import Reshape, Lambda
from keras.layers.merge import add, concatenate
from keras.models import Model
from keras.layers.recurrent import GRU
from keras.optimizers import SGD
from keras.utils.data_utils import get_file
from keras.preprocessing import image
import keras.callbacks
#from keras.metrics import Metric
from tensorflow.python.keras.metrics import Metric

OUTPUT_DIR = 'image_ocr'


def NNModel():
  # Input Parameters
  img_w = cw
  img_h = ch
  words_per_epoch = 16000
  val_split = 0.2
  val_words = int(words_per_epoch * (val_split))

  # Network parameters
  conv_filters = 16
  kernel_size = (3, 3)
  pool_size = 2
  time_dense_size = 32
  rnn_size = 512
  minibatch_size = 32


  input_shape = (None, None, 3)

  act = 'relu'
  input_data = Input(name='the_input', shape=input_shape, dtype='float32')
  inner = Conv2D(conv_filters, kernel_size, padding='same',
                  activation=act, kernel_initializer='he_normal',
                  name='conv1')(input_data)
  inner = MaxPooling2D(pool_size=(pool_size, pool_size), name='max1')(inner)
  inner = Conv2D(conv_filters, kernel_size, padding='same',
                  activation=act, kernel_initializer='he_normal',
                  name='conv2')(inner)
  inner = MaxPooling2D(pool_size=(pool_size, pool_size), name='max2')(inner)

  conv_to_rnn_dims = (img_w // (pool_size ** 2),
                      (img_h // (pool_size ** 2)) * conv_filters)
  inner = Reshape(target_shape=conv_to_rnn_dims, name='reshape')(inner)

  # cuts down input size going into RNN:
  inner = Dense(time_dense_size, activation=act, name='dense1')(inner)

  # Two layers of bidirectional GRUs
  # GRU seems to work as well, if not better than LSTM:
  gru_1 = GRU(rnn_size, return_sequences=True,
              kernel_initializer='he_normal', name='gru1')(inner)
  gru_1b = GRU(rnn_size, return_sequences=True,
                go_backwards=True, kernel_initializer='he_normal',
                name='gru1_b')(inner)
  gru1_merged = add([gru_1, gru_1b])
  gru_2 = GRU(rnn_size, return_sequences=True,
              kernel_initializer='he_normal', name='gru2')(gru1_merged)
  gru_2b = GRU(rnn_size, return_sequences=True, go_backwards=True,
                kernel_initializer='he_normal', name='gru2_b')(gru1_merged)

  # transforms RNN output to character activations:
  inner = Dense(20, kernel_initializer='he_normal', name='dense2')(concatenate([gru_2, gru_2b]))
  y_pred = Activation('softmax', name='softmax')(inner)



  labels = Input(name='the_labels', shape=[max_word_length_track], dtype='float32')
  input_length = Input(name='input_length', shape=[1], dtype='int64')
  label_length = Input(name='label_length', shape=[1], dtype='int64')


  # model0 = Model(
  #     inputs=[input_data, labels, input_length, label_length],
  #     outputs=y_pred
  # )


  def ctc_lambda_func(args):
      y_pred, labels, input_length, label_length = args
      # the 2 is critical here since the first couple outputs of the RNN
      # tend to be garbage:
      y_pred = y_pred[:, 2:, :]
      return K.ctc_batch_cost(labels, y_pred, input_length, label_length)

  # Keras doesn't currently support loss funcs with extra parameters
  # so CTC loss is implemented in a lambda layer
  loss_out = Lambda(
      ctc_lambda_func, output_shape=(1,),
      name='ctc')([y_pred, labels, input_length, label_length])

  # clipnorm seems to speeds up convergence
  sgd = SGD(lr=0.02, decay=1e-6, momentum=0.9, nesterov=True, clipnorm=5)

  model0 = Model(
          inputs=[input_data, labels, input_length, label_length],
          outputs=loss_out
      )

  model0.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer='adam')


  model0.summary()
  train_model = model0

  prediction_model = Model(inputs=input_data, outputs=y_pred)

  return train_model, prediction_model


model0, _ = NNModel()

In [ ]:
len(train_L_id[0])

In [ ]:
max_word_length_track

In [ ]:
batch_size = 25 #a divisor of training length
epochs = 20

input_length = np.array([ [1] for i in range(n) ])
label_length = np.array([ [1] for i in range(n) ])

metrics0 = model0.fit(
          x=[
              train_F, 
              train_L_id, 
              input_length,
              label_length,
            ], 
          y = np.zeros(len(train_F)),
          # batch_size=batch_size, 
          epochs = epochs, 
          # validation_data = ([val_img, val_padded_txt, val_input_length, val_label_length], val_padded_txt),
          # verbose = 2,
          # callbacks = callbacks_list
)

In [ ]:
model0.save_weights( './weights2.h5' )

In [ ]:
[print(i.shape, i.dtype) for i in model0.inputs]
# [print(o.shape, o.dtype) for o in model0.outputs]
# [print(l.name, l.input_shape, l.dtype) for l in model0.layers]

### Predict

In [ ]:
# pred = model0.predict(
#     [train_F, train_L_id, np.array([1]), np.array([1])]
# )

In [ ]:
# pred

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
img = train_F[0]
img = img.astype(np.uint8) 
plt.title(train_L[0])
plt.imshow(img * 255)

In [ ]:
_, model_infer = NNModel()
model_infer.load_weights("./weights2.h5")

In [ ]:
model_infer.summary()

In [ ]:
len(train_F)

In [ ]:
# sample = train_F[0].reshape(1, 100, 780, 3)
yhat = model_infer.predict(train_F[0].reshape(1, ch, cw, 3))

In [ ]:
yhat.shape

In [ ]:
len(yhat)

In [ ]:
yhat[0][211][10]

In [ ]:
np.argmax(yhat, axis=1)

In [ ]:
len(alphabet)

In [ ]:
for i, y_ in enumerate(yhat[0]):
  print(y_)